# Real Time Emotion Recognition using CNN and Transfer Learning:

# 1) Environment Check :¶

In [1]:
!python --version

Python 3.11.13


In [1]:
import tensorflow
print(tensorflow.__version__)

2.19.0


In [1]:
import torch, torchvision
import os
print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)
print("CUDA Available:", torch.cuda.is_available())

Torch: 2.7.1+cpu
Torchvision: 0.22.1+cpu
CUDA Available: False


# 2) Importing Libraries :

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import os
import numpy as np
import random

# 3) Creating data from video :

In [5]:
def add_salt_pepper_noise(image, amount=0.004):
    noisy = image.copy()
    num_salt = np.ceil(amount * image.size * 0.5).astype(int)
    num_pepper = np.ceil(amount * image.size * 0.5).astype(int)

    for k in range(num_salt):
        i = random.randint(0, image.shape[0] - 1)
        j = random.randint(0, image.shape[1] - 1)
        noisy[i, j] = [255, 255, 255]

    for k in range(num_pepper):
        i = random.randint(0, image.shape[0] - 1)
        j = random.randint(0, image.shape[1] - 1)
        noisy[i, j] = [0, 0, 0]

    return noisy

In [6]:
def random_contrast(image, lower=0.5, upper=1.5):
    factor = random.uniform(lower, upper)
    return cv2.convertScaleAbs(image, alpha=factor, beta=0)

In [7]:
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, matrix, (w, h))

In [10]:
def capture_images(label, cam_index=0, output_base_dir=r'C:\Users\SONAM\Desktop\DL\CNN Final Project\Train_Val_Images', max_images=1002):
    detector = cv2.CascadeClassifier(r"C:\Users\SONAM\Desktop\DL\CNN Final Project\haarcascade_frontalface_default.xml")
    source = cv2.VideoCapture(cam_index)

    class_dir = os.path.join(output_base_dir, label)
    os.makedirs(class_dir, exist_ok=True)   #output sub folders

    existing_images = len(os.listdir(class_dir))
    c = existing_images

    
    while c < max_images:
        ret, frame = source.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        result = detector.detectMultiScale(gray, scaleFactor=1.4, minNeighbors=4)

        if len(result) >= 1:
            x, y, w, h = result[0]
            cropped = frame[y:y+h, x:x+w]

            aug_images = [
                cropped,
                cv2.flip(cropped, 1),
                cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY),
                cv2.GaussianBlur(cropped, (3, 3), 3),
                add_salt_pepper_noise(cropped),
                random_contrast(cropped),
                rotate_image(cropped, 30),
                rotate_image(cropped, -30)
            ]

            for img in aug_images:
                if len(img.shape) == 2:  # grayscale to BGR for saving
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
                if c < max_images:
                    filename = os.path.join(class_dir, f"{label}_{c}.jpg")
                    cv2.imwrite(filename, img)
                    c += 1
                else:
                    break

        cv2.imshow("Face Capture", frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    source.release()
    cv2.destroyAllWindows()
    #print(f"Done. Total images in '{class_dir}': {c}")

In [14]:
capture_images("Sleep", cam_index=0)

In [11]:
capture_images("Angry", cam_index=0)

In [9]:
capture_images("Smile", cam_index=0)

Starting capture for class: Smile. Already 0 images exist.
Done. Total images in 'C:\Users\SONAM\Desktop\DL\CNN Final Project\Train_Val_Images\Smile': 1002


In [12]:
capture_images("Sad", cam_index=0)

In [13]:
capture_images("Surprise", cam_index=0)

# 4) Performed preprocessing, building the model , training the model on train data and evaluation in colab with pytorch

# 5) Inference on Real time Emotion recognition :

In [3]:
import torch
import torch.nn as nn
from torchvision import models

# Defining model architecture exactly as in training
def get_emotion_model(num_classes=5):
    model = models.resnet18(pretrained=False)  # pretrained=False because we load weights
    for param in model.parameters():
        param.requires_grad = False

    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 256),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, num_classes)
    )
    return model

In [4]:
model = get_emotion_model(num_classes=5)  # use same num_classes as during training

# Load the state_dict
model.load_state_dict(torch.load(
    r"C:\Users\SONAM\Desktop\DL\CNN Final Project\emotion_model.pth", 
    map_location=torch.device('cpu'))
)

model.eval()  # Ready for inference

C:\Users\SONAM\anaconda3\envs\cnn_project\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\SONAM\anaconda3\envs\cnn_project\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
classes = ['Angry','Sad','Sleep','Smile', 'Surprise']

# Define input transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read() #reading the frames from video
    if not ret:
        break

    # Convert BGR to RGB for PIL
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img)

    # Preprocess
    input_tensor = transform(img_pil).unsqueeze(0)

    with torch.no_grad():
        output = model(input_tensor)
        pred = torch.argmax(output, dim=1).item()
        label = classes[pred]

    # Display prediction
    cv2.putText(frame, f'Emotion: {label}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Live Emotion Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()